<a href="https://colab.research.google.com/github/mmilannaik/Spark/blob/main/Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
!tar xf spark-3.5.6-bin-hadoop3.tgz

# spark latest version https://downloads.apache.org/spark/


In [2]:
!pip install -q pyspark


In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.6-bin-hadoop3"


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColabSparkDemo") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

spark


In [5]:
print("Spark Version:", spark.version)


Spark Version: 3.5.6


# Spark Handson

In [6]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [7]:
# create a dataframe
emp = spark.createDataFrame(data = emp_data, schema = emp_schema)

In [8]:
# check the no of RDDs
emp.rdd.getNumPartitions()

2

In [9]:
# Show data(ACTION)
emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [ ]:
#write about transformation
emp_final = emp.where("salary > 50000")

In [ ]:
# validate the partitions
emp_final.rdd.getNumPartitions()

2

In [ ]:
emp_final.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|
|        013|          106|    Brian Kim| 45|  Male| 75000|2011-07-01|
|        015|          106|  Michael Lee| 37|  Male| 63000|2014-09-30|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|
|     

# Transformation-P1

In [ ]:
#schema for emop
# this is the native way of storing schema
emp.schema

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [ ]:
# schema for emp
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [ ]:
# small example for schema
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
schema_string = "name string,age int"

schema_spark = StructType([
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True)
])

In [ ]:
# columns and expression
from pyspark.sql.functions import col,expr
emp.salary

Column<'salary'>

In [ ]:
emp["salary"]

Column<'salary'>

In [ ]:
# select columns
emp_filtered = emp.select(col("employee_id"),expr("name"),emp.age,emp.salary)

In [ ]:
# show actions
emp_filtered.show()

+-----------+-------------+---+------+
|employee_id|         name|age|salary|
+-----------+-------------+---+------+
|        001|     John Doe| 30| 50000|
|        002|   Jane Smith| 25| 45000|
|        003|    Bob Brown| 35| 55000|
|        004|    Alice Lee| 28| 48000|
|        005|    Jack Chan| 40| 60000|
|        006|    Jill Wong| 32| 52000|
|        007|James Johnson| 42| 70000|
|        008|     Kate Kim| 29| 51000|
|        009|      Tom Tan| 33| 58000|
|        010|     Lisa Lee| 27| 47000|
|        011|   David Park| 38| 65000|
|        012|   Susan Chen| 31| 54000|
|        013|    Brian Kim| 45| 75000|
|        014|    Emily Lee| 26| 46000|
|        015|  Michael Lee| 37| 63000|
|        016|  Kelly Zhang| 30| 49000|
|        017|  George Wang| 34| 57000|
|        018|    Nancy Liu| 29| 50000|
|        019|  Steven Chen| 36| 62000|
|        020|    Grace Kim| 32| 53000|
+-----------+-------------+---+------+



In [ ]:
#using expr for select
# select employee_id
emp_casted = emp_filtered.select(expr("employee_id as emp_id"), emp.name,expr("cast(age as int) as age"),emp.salary)


In [ ]:
# show action
emp_casted.show()

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   001|     John Doe| 30| 50000|
|   002|   Jane Smith| 25| 45000|
|   003|    Bob Brown| 35| 55000|
|   004|    Alice Lee| 28| 48000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   008|     Kate Kim| 29| 51000|
|   009|      Tom Tan| 33| 58000|
|   010|     Lisa Lee| 27| 47000|
|   011|   David Park| 38| 65000|
|   012|   Susan Chen| 31| 54000|
|   013|    Brian Kim| 45| 75000|
|   014|    Emily Lee| 26| 46000|
|   015|  Michael Lee| 37| 63000|
|   016|  Kelly Zhang| 30| 49000|
|   017|  George Wang| 34| 57000|
|   018|    Nancy Liu| 29| 50000|
|   019|  Steven Chen| 36| 62000|
|   020|    Grace Kim| 32| 53000|
+------+-------------+---+------+



In [ ]:
#show schema
emp_casted.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: string (nullable = true)



In [ ]:
# use of selectExpr
emp_casted_1 = emp_filtered.selectExpr("employee_id as emp_id","name","cast(age as int) as age","salary")

In [ ]:
emp_casted_1.show()

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   001|     John Doe| 30| 50000|
|   002|   Jane Smith| 25| 45000|
|   003|    Bob Brown| 35| 55000|
|   004|    Alice Lee| 28| 48000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   008|     Kate Kim| 29| 51000|
|   009|      Tom Tan| 33| 58000|
|   010|     Lisa Lee| 27| 47000|
|   011|   David Park| 38| 65000|
|   012|   Susan Chen| 31| 54000|
|   013|    Brian Kim| 45| 75000|
|   014|    Emily Lee| 26| 46000|
|   015|  Michael Lee| 37| 63000|
|   016|  Kelly Zhang| 30| 49000|
|   017|  George Wang| 34| 57000|
|   018|    Nancy Liu| 29| 50000|
|   019|  Steven Chen| 36| 62000|
|   020|    Grace Kim| 32| 53000|
+------+-------------+---+------+



In [ ]:
# filter age >30
emp_final = emp_casted.select("emp_id","name","age","salary").where("age > 30")

In [ ]:
emp_final.show()

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   003|    Bob Brown| 35| 55000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   009|      Tom Tan| 33| 58000|
|   011|   David Park| 38| 65000|
|   012|   Susan Chen| 31| 54000|
|   013|    Brian Kim| 45| 75000|
|   015|  Michael Lee| 37| 63000|
|   017|  George Wang| 34| 57000|
|   019|  Steven Chen| 36| 62000|
|   020|    Grace Kim| 32| 53000|
+------+-------------+---+------+



In [ ]:
#write back
emp_final.write.format("csv").save( "emp.csv")

In [ ]:
#Bonus Tip
schema_str = "name string, age int"
from pyspark.sql.types import _parse_datatype_string

schema_spark = _parse_datatype_string(schema_str)
schema_spark

StructType([StructField('name', StringType(), True), StructField('age', IntegerType(), True)])

# Transformation P2

In [ ]:
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [10]:
# cast columns
from pyspark.sql.functions import col,cast

emp_casted = emp.select("employee_id","name","age",col("salary").cast("double"))


In [11]:
emp_casted.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)



In [12]:
# Adding columns
emp_taxed = emp_casted.withColumn("tax",col("salary")*0.2)

In [13]:
emp_taxed.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [14]:
#literals
# add new columsn with static value
from pyspark.sql.functions import lit

emp_new_cols = emp_taxed.withColumn("ColumnOne",lit(1)).withColumn("columntwo",lit("three"))

In [15]:
emp_new_cols.show()

+-----------+-------------+---+-------+-------+---------+---------+
|employee_id|         name|age| salary|    tax|ColumnOne|columntwo|
+-----------+-------------+---+-------+-------+---------+---------+
|        001|     John Doe| 30|50000.0|10000.0|        1|    three|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|    three|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|    three|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|    three|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|    three|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|    three|
|        007|James Johnson| 42|70000.0|14000.0|        1|    three|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|    three|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|    three|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|    three|
|        011|   David Park| 38|65000.0|13000.0|        1|    three|
|        012|   Susan Chen| 31|54000.0|10800.0| 

In [16]:
# renaming cols
emp_1 = emp_new_cols.withColumnRenamed("employee_id","emp_id")

In [17]:
emp_1.show()

+------+-------------+---+-------+-------+---------+---------+
|emp_id|         name|age| salary|    tax|ColumnOne|columntwo|
+------+-------------+---+-------+-------+---------+---------+
|   001|     John Doe| 30|50000.0|10000.0|        1|    three|
|   002|   Jane Smith| 25|45000.0| 9000.0|        1|    three|
|   003|    Bob Brown| 35|55000.0|11000.0|        1|    three|
|   004|    Alice Lee| 28|48000.0| 9600.0|        1|    three|
|   005|    Jack Chan| 40|60000.0|12000.0|        1|    three|
|   006|    Jill Wong| 32|52000.0|10400.0|        1|    three|
|   007|James Johnson| 42|70000.0|14000.0|        1|    three|
|   008|     Kate Kim| 29|51000.0|10200.0|        1|    three|
|   009|      Tom Tan| 33|58000.0|11600.0|        1|    three|
|   010|     Lisa Lee| 27|47000.0| 9400.0|        1|    three|
|   011|   David Park| 38|65000.0|13000.0|        1|    three|
|   012|   Susan Chen| 31|54000.0|10800.0|        1|    three|
|   013|    Brian Kim| 45|75000.0|15000.0|        1|   

In [18]:
# column names with spaces
emp_2 = emp_new_cols.withColumnRenamed("columntwo","column two")

In [19]:
emp_2.show()

+-----------+-------------+---+-------+-------+---------+----------+
|employee_id|         name|age| salary|    tax|ColumnOne|column two|
+-----------+-------------+---+-------+-------+---------+----------+
|        001|     John Doe| 30|50000.0|10000.0|        1|     three|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|     three|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|     three|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|     three|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|     three|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|     three|
|        007|James Johnson| 42|70000.0|14000.0|        1|     three|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|     three|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|     three|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|     three|
|        011|   David Park| 38|65000.0|13000.0|        1|     three|
|        012|   Susan Chen| 31|540

In [21]:
# remove cols
emp_dropped = emp_new_cols.drop("columntwo","columnOne")

In [22]:
emp_dropped.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [23]:
# Filter
emp_filtered = emp_dropped.where("tax > 10000")
emp_filtered.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        003|    Bob Brown| 35|55000.0|11000.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        019|  Steven Chen| 36|62000.0|12400.0|
|        020|    Grace Kim| 32|53000.0|10600.0|
+-----------+-------------+---+-------+-------+



In [24]:
# limit data
emp_limit = emp_filtered.limit(5)

In [25]:
emp_limit.show(2)

+-----------+---------+---+-------+-------+
|employee_id|     name|age| salary|    tax|
+-----------+---------+---+-------+-------+
|        003|Bob Brown| 35|55000.0|11000.0|
|        005|Jack Chan| 40|60000.0|12000.0|
+-----------+---------+---+-------+-------+
only showing top 2 rows



In [27]:
#Bonus Tip
columns = {
    "tax": col("salary") *0.2,
    "oneNumber" : lit(1),
    "columnTwo" : lit("two")

}

emp_final = emp.withColumns(columns)

In [28]:
emp_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------+---------+---------+
|employee_id|department_id|         name|age|gender|salary| hire_date|    tax|oneNumber|columnTwo|
+-----------+-------------+-------------+---+------+------+----------+-------+---------+---------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|10000.0|        1|      two|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15| 9000.0|        1|      two|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|11000.0|        1|      two|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30| 9600.0|        1|      two|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|12000.0|        1|      two|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|10400.0|        1|      two|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|14000.0|        1|      two|
|        0

# Stings,Dates & Null

In [29]:
# case when
# case gender male then m when gender female F
from pyspark.sql.functions import when

emp_gender_fixed = emp.withColumn("new_gender",when(col("gender")== 'Male','M').
                                  when(col('gender')=='Female','F').
                                  otherwise(None))
emp_gender_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|         M|
|        010|   

In [30]:
# case when
# case gender male then m when gender female F
from pyspark.sql.functions import when,col,expr

emp_gender_fixed1 = emp.withColumn("new_gender",expr("case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end"))

emp_gender_fixed1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|         M|
|        010|   

In [33]:
# Replace in Strings
# select employee_id, name, replace as new_name,age, salary, gender, new_gender, hire_date

from pyspark.sql.functions import regexp_replace

emp_name_fixed = emp_gender_fixed.withColumn("new_name",regexp_replace(col("name"),"J","Z"))

In [34]:
emp_name_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim